## Read Grid Files

* Read CMG-style grid files.
* Count number of values.
* Output in n*x style.

In [7]:
from pathlib import Path

In [89]:
test_file1 = Path(r'..\gridfiles\PERMK.geo').resolve()
print(f'File {test_file1.name} exists => {test_file1.is_file()}')

test_file2 = Path(r'..\gridfiles\RTYPE.geo').resolve()
print(f'File {test_file2.name} exists => {test_file2.is_file()}')

File PERMK.geo exists => True
File RTYPE.geo exists => True


In [87]:
def parse_grid_file(file_path):
    data = {'command':'', 'comments':list(), 'values':list()}

    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False
        
    def int_or_float(s):
        try:
            x = int(s)
            return x
        except ValueError:
            try:
                x = float(s)
                return x
            except ValueError:
                raise ValueError("Input string is not a valid number")

    def is_line_with_values(line):
        parts = line.split(' ')
        parts = parts[0].split('*')
        return is_number(parts[0])

    with file_path.open(mode='r') as file:
        for line in file:
            line = line.strip()
            if line == '':
                pass
            elif line.startswith('**'):
                data['comments'].append(line)
            elif not is_line_with_values(line):
                if data['command'] != '':
                    raise ValueError("Input file is not a grid file!")
                data['command'] = line
            else:
                parts = line.split('**')
                parts = parts[0].strip().split(' ')
                for n in parts:
                    value = n.split('*')
                    if len(value) == 1:
                        data['values'].append(int_or_float(value[0]))
                    else:
                        data['values'].extend([int_or_float(value[1]) for i in range(int_or_float(value[0]))])

    if len(data['values']) == 0:
        raise ValueError("Input file is not a grid file!")
    return data

In [63]:
_Max_line = 80

def write_grid_file(data, file_path):
    lines = [c for c in data['comments']]
    lines.append(data['command'])

    current_line = ''
    def add_line(s, current_line):
        if len(current_line) + len(s) > _Max_line:
            lines.append(current_line)
            current_line = ''
        return current_line + s

    current_value = 0
    n_values = 0
    for value in data['values']:
        if value == current_value:
            n_values += 1
        else:
            if n_values == 1:
                current_line = add_line(s=' '+str(current_value), current_line=current_line)
            elif n_values > 1:
                current_line = add_line(s=' '+str(n_values)+'*'+str(current_value), current_line=current_line)
            current_value = value
            n_values = 1

    if n_values == 1:
        lines.append(current_line+' '+str(current_value))
    elif n_values > 1:
        lines.append(current_line+' '+str(n_values)+'*'+str(current_value))

    content = '\n'.join(lines)
    file_path.write_text(content)

In [81]:
def count_values_grid_file(file_path):
    try:
        data = parse_grid_file(file_path)
        return len(data['values'])
    except ValueError:
        return 0

In [91]:
def rewrite_grid_file(file_path, new_suffix=None, verbose=False):
    if file_path.is_dir():
        for file in file_path.iterdir():
            if file.is_file():
                rewrite_grid_file(file, new_suffix, True)
    else:
        try:
            if verbose:
                print(f'Parsing {file_path.name}...')
            data = parse_grid_file(file_path)
            out_file_path = file_path
            if new_suffix is not None:
                out_file_path = file_path.with_suffix(new_suffix)
            if verbose:
                print(f'Exporting {out_file_path.name}...')
            write_grid_file(data=data, file_path=out_file_path)
        except ValueError:
            print(f'Could not parse {file_path.name}')

In [92]:
data1 = parse_grid_file(test_file1)
write_grid_file(data=data1, file_path=test_file1.with_suffix('.new'))
print(f'Original file: {count_values_grid_file(test_file1)} values')
print(f'New file:      {count_values_grid_file(test_file1.with_suffix('.new'))} values')

Original file: 533403 values
New file:      533403 values


In [93]:
data2 = parse_grid_file(test_file2)
write_grid_file(data=data2, file_path=test_file2.with_suffix('.new'))
print(f'Original file: {count_values_grid_file(test_file2)} values')
print(f'New file:      {count_values_grid_file(test_file2.with_suffix('.new'))} values')

Original file: 533403 values
New file:      533403 values


In [94]:
rewrite_grid_file(test_file1.parent)

Parsing PERMK.geo...
Exporting PERMK.geo...
Parsing PERMK.new...
Exporting PERMK.new...
Parsing RTYPE.geo...
Exporting RTYPE.geo...
Parsing RTYPE.new...
Exporting RTYPE.new...
Parsing RTYPE_error.geo...
Could not parse RTYPE_error.geo
Parsing RTYPE_error2.geo...
Could not parse RTYPE_error2.geo
